In [1]:
from keras import models, layers, optimizers
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing import image
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, shutil

In [2]:
#pip install tensorflow

ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

xarray 2022.6.0 requires pandas>=1.2, but you'll have pandas 1.1.3 which is incompatible.
pingouin 0.5.2 requires scipy>=1.7, but you'll have scipy 1.5.2 which is incompatible.
pingouin 0.5.2 requires statsmodels>=0.13, but you'll have statsmodels 0.12.0 which is incompatible.


  Created wheel for jax: filename=jax-0.4.6-py3-none-any.whl size=1432708 sha256=555818bd8edba6f9c55b16f45cc7b4b97f8b58ebd3b21383748d105474645e97
  Stored in directory: c:\users\shriy\appdata\local\pip\cache\wheels\f2\db\3a\c9d0d455324439943bdc616f148bf02c4e57b2b995c39cedad
Successfully built jax
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.2
    Uninstalling numpy-1.19.2:
      Successfully uninstalled numpy-1.19.2
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.18.0
    Uninstalling protobuf-3.18.0:
      Successfully uninstalled protobuf-3.18.0
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 2.0.0
    Uninstalling importlib-metadata-2.0.0:
      Successfully uninstalled importlib-metadata-2.0.0


In [3]:
#Load train and test data as dataframes 
train_dataset = pd.read_csv('./dataset/train.csv')
test_dataset = pd.read_csv('./dataset/test.csv')

print("Length of training data is: ", train_dataset.shape[0])
print("Length of testing data is: ", test_dataset.shape[0])

print("\nSummary of first 10 rows of train data:")
train_dataset.head(10)


Length of training data is:  3662
Length of testing data is:  1928

Summary of first 10 rows of train data:


,id_code,diagnosis
0,000c1434d8d7,2
1,001639a390f0,4
2,0024cdab0c1e,1
3,002c21358ce6,0
4,005b95c28852,0
5,0083ee8054ee,4
6,0097f532ac9f,0
7,00a8624548a9,2
8,00b74780d31d,2
9,00cb6555d108,1


In [21]:
validation_dataset =train_dataset.iloc[:1200,:]
print(validation_dataset.shape[0])
print(validation_dataset)

train_dataset = train_dataset.iloc[1200:,:]
print(train_dataset.shape[0])
print(train_dataset)


1200
           id_code  diagnosis
0     000c1434d8d7          2
1     001639a390f0          4
2     0024cdab0c1e          1
3     002c21358ce6          0
4     005b95c28852          0
...            ...        ...
1195  5299a532f0e0          0
1196  52ae917fcea4          1
1197  52dbec057cc8          2
1198  52ddde91a349          0
1199  52edbe29d655          0

[1200 rows x 2 columns]
2462
           id_code  diagnosis
1200  530d78467615          2
1201  531b39880c32          1
1202  5321ab64f9ea          0
1203  53273d664cd8          0
1204  5327f88a1919          2
...            ...        ...
3657  ffa47f6a7bf4          2
3658  ffc04fed30e6          0
3659  ffcf7b45f213          2
3660  ffd97f8cd5aa          0
3661  ffec9a18a3ce          2

[2462 rows x 2 columns]


In [22]:
#Split the training data into the 5 classes
class_0_train=[]
class_1_train=[]
class_2_train=[]
class_3_train=[]
class_4_train=[]

for diag0 in train_dataset[train_dataset['diagnosis']==0]['id_code']:
    class_0_train.append(diag0)
for diag1 in train_dataset[train_dataset['diagnosis']==1]['id_code']:
    class_1_train.append(diag1)
for diag2 in train_dataset[train_dataset['diagnosis']==2]['id_code']:
    class_2_train.append(diag2)
for diag3 in train_dataset[train_dataset['diagnosis']==3]['id_code']:
    class_3_train.append(diag3)
for diag4 in train_dataset[train_dataset['diagnosis']==4]['id_code']:
    class_4_train.append(diag4)

'''
Note, if we wanna create validation sets then create 2 lists (class_0_train and class_0_val) 
and do a split on class_0 list from above
'''

#add .png extension to the end of each id which will be used later
for i in range(0, len(class_0_train)):
    class_0_train[i] = class_0_train[i]+".png"

for i in range(0, len(class_1_train)):
    class_1_train[i] = class_1_train[i]+".png"

for i in range(0, len(class_2_train)):
    class_2_train[i] = class_2_train[i]+".png"

for i in range(0, len(class_3_train)):
    class_3_train[i] = class_3_train[i]+".png"

for i in range(0, len(class_4_train)):
    class_4_train[i] = class_4_train[i]+".png"

print("Length of training data in class 0:", len(class_0_train))
print("Length of training data in class 1:", len(class_1_train))
print("Length of training data in class 2:", len(class_2_train))
print("Length of training data in class 3:", len(class_3_train))
print("Length of training data in class 4:", len(class_4_train))

class_0_train

Length of training data in class 0: 1249
Length of training data in class 1: 244
Length of training data in class 2: 641
Length of training data in class 3: 126
Length of training data in class 4: 202


['5321ab64f9ea.png',
 '53273d664cd8.png',
 '53327edb9e4d.png',
 '535682537302.png',
 '537e5c578f40.png',
 '53c874dbc594.png',
 '53ddae6a619e.png',
 '54038e56131d.png',
 '540e4973829e.png',
 '542964865b1e.png',
 '54334705a34d.png',
 '5445255635f0.png',
 '549381330191.png',
 '54b322c66d01.png',
 '54bdcdecd8f3.png',
 '54f57cf26126.png',
 '5511f114e7ee.png',
 '555d0bef3c5b.png',
 '55968f0e63c4.png',
 '55f7f018c61c.png',
 '565f3404f9b2.png',
 '568455854a11.png',
 '571bbdbf585e.png',
 '5728b8aa98ef.png',
 '573ea80a53be.png',
 '57a5e4274275.png',
 '57a710de68a4.png',
 '57db4781e7ec.png',
 '57f5ad4b5b29.png',
 '58059e73d2d4.png',
 '5814cbd2e9bf.png',
 '586f5c56081e.png',
 '5889a0c75cac.png',
 '58a9e0d7f7af.png',
 '58af2c054ced.png',
 '58c12863f33d.png',
 '58eb3809f456.png',
 '58f07741ee3b.png',
 '596f4fdb0004.png',
 '598b8f5b3822.png',
 '59928f999ae7.png',
 '599498e9e4bc.png',
 '5995321563b7.png',
 '59bd19c1c5bb.png',
 '5a0fe0ee4301.png',
 '5a179c123fd8.png',
 '5a27b9b2a9c1.png',
 '5a444c32cd9

In [23]:
#Split the validation data into the 5 classes
class_0_val=[]
class_1_val=[]
class_2_val=[]
class_3_val=[]
class_4_val=[]

for diag0 in validation_dataset[validation_dataset['diagnosis']==0]['id_code']:
    class_0_val.append(diag0)
for diag1 in validation_dataset[validation_dataset['diagnosis']==1]['id_code']:
    class_1_val.append(diag1)
for diag2 in validation_dataset[validation_dataset['diagnosis']==2]['id_code']:
    class_2_val.append(diag2)
for diag3 in validation_dataset[validation_dataset['diagnosis']==3]['id_code']:
    class_3_val.append(diag3)
for diag4 in validation_dataset[validation_dataset['diagnosis']==4]['id_code']:
    class_4_val.append(diag4)

'''
Note, if we wanna create validation sets then create 2 lists (class_0_train and class_0_val) 
and do a split on class_0 list from above
'''

#add .png extension to the end of each id which will be used later
for i in range(0, len(class_0_val)):
    class_0_val[i] = class_0_val[i]+".png"

for i in range(0, len(class_1_val)):
    class_1_val[i] = class_1_val[i]+".png"

for i in range(0, len(class_2_val)):
    class_2_val[i] = class_2_val[i]+".png"

for i in range(0, len(class_3_val)):
    class_3_val[i] = class_3_val[i]+".png"

for i in range(0, len(class_4_val)):
    class_4_val[i] = class_4_val[i]+".png"

print("Length of training data in class 0:", len(class_0_val))
print("Length of training data in class 1:", len(class_1_val))
print("Length of training data in class 2:", len(class_2_val))
print("Length of training data in class 3:", len(class_3_val))
print("Length of training data in class 4:", len(class_4_val))

class_0_val

Length of training data in class 0: 556
Length of training data in class 1: 126
Length of training data in class 2: 358
Length of training data in class 3: 67
Length of training data in class 4: 93


['002c21358ce6.png',
 '005b95c28852.png',
 '0097f532ac9f.png',
 '00cc2b75cddd.png',
 '00f6c1be5a33.png',
 '0125fbd2e791.png',
 '014508ccb9cb.png',
 '0151781fe50b.png',
 '0182152c50de.png',
 '01d9477b1171.png',
 '01f7bb8be950.png',
 '0212dd31f623.png',
 '022f820027b8.png',
 '0231642cf1c2.png',
 '0232dfea7547.png',
 '02358b47ea89.png',
 '02cd34a85b24.png',
 '02da652c74b8.png',
 '0304bedad8fe.png',
 '03b373718013.png',
 '0423237770a7.png',
 '04579e31e4be.png',
 '04aef84a2cc1.png',
 '04efb1a284cc.png',
 '050bb1eafa76.png',
 '05113073b268.png',
 '05195a3db5e2.png',
 '05339950962e.png',
 '054b1b305160.png',
 '0551676cc2aa.png',
 '05b1bb2bdb81.png',
 '05e9126dfa5c.png',
 '060e00d1e2ab.png',
 '06586082a24d.png',
 '06be1092a062.png',
 '0709652336e2.png',
 '070d4ce5fd90.png',
 '07596907347b.png',
 '0773a1c326ad.png',
 '07751b94a88a.png',
 '0790515cf5af.png',
 '07a0be6b347f.png',
 '07a2b8cabf6b.png',
 '07e827469099.png',
 '07f5d7baf907.png',
 '08037e4490e5.png',
 '084c02cf077f.png',
 '0851d6a6958

In [25]:
#Path to directory where the images are
train_dataset_dir = './dataset/train_images/'
test_dataset_dir = './dataset/test_images/'

#Directory where the new labelled training dataset will be created and stored
new_dataset = './new_dataset'
os.mkdir(new_dataset)
train_dir = os.path.join(new_dataset, 'train')
os.mkdir(train_dir)
val_dir= os.path.join(new_dataset, 'val')
os.mkdir(val_dir)
#Directories with training images for each class
train_class0 = os.path.join(train_dir, 'class0')
os.mkdir(train_class0)
train_class1 = os.path.join(train_dir, 'class1')
os.mkdir(train_class1)
train_class2 = os.path.join(train_dir, 'class2')
os.mkdir(train_class2)
train_class3 = os.path.join(train_dir, 'class3')
os.mkdir(train_class3)
train_class4 = os.path.join(train_dir, 'class4')
os.mkdir(train_class4)
#Directories with validation images for each class
val_class0 = os.path.join(val_dir, 'class0')
os.mkdir(val_class0)
val_class1 = os.path.join(val_dir, 'class1')
os.mkdir(val_class1)
val_class2 = os.path.join(val_dir, 'class2')
os.mkdir(val_class2)
val_class3 = os.path.join(val_dir, 'class3')
os.mkdir(val_class3)
val_class4 = os.path.join(val_dir, 'class4')
os.mkdir(val_class4)



In [26]:
#Use labels from class list to add the images to their appropriate class directory

#Class 0:
for k in class_0_train:
    src = os.path.join(train_dataset_dir, k)
    dst = os.path.join(train_class0, k)
    shutil.copyfile(src, dst)

#Class 1:
for k in class_1_train:
    src = os.path.join(train_dataset_dir, k)
    dst = os.path.join(train_class1, k)
    shutil.copyfile(src, dst)

#Class 2:
for k in class_2_train:
    src = os.path.join(train_dataset_dir, k)
    dst = os.path.join(train_class2, k)
    shutil.copyfile(src, dst)

#Class 3:
for k in class_3_train:
    src = os.path.join(train_dataset_dir, k)
    dst = os.path.join(train_class3, k)
    shutil.copyfile(src, dst)

#Class 4:
for k in class_4_train:
    src = os.path.join(train_dataset_dir, k)
    dst = os.path.join(train_class4, k)
    shutil.copyfile(src, dst)
print("done")

done


In [27]:
#Use labels from class list to add the images to their appropriate class directory

#Class 0:
for k in class_0_val:
    src = os.path.join(train_dataset_dir, k)
    dst = os.path.join(val_class0, k)
    shutil.copyfile(src, dst)

#Class 1:
for k in class_1_val:
    src = os.path.join(train_dataset_dir, k)
    dst = os.path.join(val_class1, k)
    shutil.copyfile(src, dst)

#Class 2:
for k in class_2_val:
    src = os.path.join(train_dataset_dir, k)
    dst = os.path.join(val_class2, k)
    shutil.copyfile(src, dst)

#Class 3:
for k in class_3_val:
    src = os.path.join(train_dataset_dir, k)
    dst = os.path.join(val_class3, k)
    shutil.copyfile(src, dst)

#Class 4:
for k in class_4_val:
    src = os.path.join(train_dataset_dir, k)
    dst = os.path.join(val_class4, k)
    shutil.copyfile(src, dst)
print("done")

done


In [52]:
#Build model with convolution layers, pooling, and normalization
model = models.Sequential()
model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(32,32,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Flatten())
model.add(layers.Dense(5, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 74, 74, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                        